In [85]:
import pandas
from matplotlib import pyplot as plt
import numpy as np
import geopandas
import urllib
import json
from sklearn.neighbors import NearestNeighbors

## Process Data

In [2]:
base_url = "http://data.rcc-acis.org/"
#######################################
#Acis WebServices functions
#######################################
def make_request(url,params) :
    data = json.dumps(params)
    data_bytes = bytes(data, 'utf-8') # one weird trick
    
    req = urllib.request.Request(url,
    data=data_bytes,
    headers={"Content-Type":"application/json"})
    try:
        response = urllib.request.urlopen(req)
        return json.loads(response.read())
    except urllib.request.HTTPError as error:
        if error.code == 400 : print(error.msg)
    print(urllib.request.HTTPError)

def GridData(params, server="GridData"):
    return make_request(base_url+server, params)

In [67]:
# see 
# https://www.rcc-acis.org/docs_webservices.html 
# (section IV onwards) for parameter values.

params = {
    "state":["CO"],
    "sdate":"20200101", # low-tech YYYYMMDD
    "edate":"20201231",
    "grid":1,
    "output":"json",
    "elems":[{"name":"pcpn","smry":"sum","smry_only":1}],  # annual summed precipitation
    "meta":["ll"]
}

#Obtain precipitation data
precip_data = GridData(params, server="GridData")

params = {
    "state":["CO"],
    "sdate":"20200101", # low-tech YYYYMMDD
    "edate":"20201231",
    "grid":1,
    "output":"json",
    "elems":[{"name":"maxt","smry":"max","smry_only":1}],  # annual max temp
    # TODO: compbine the maxtemp (maybe average?) and summed precipitaiton better?
    "meta":["ll", "elev"]
}

#Obtain temperature and elevation data
temp_elev_data = GridData(params, server="GridData")

In [65]:
print("Precipitation Data")
d = pandas.DataFrame(dict([ (k,pandas.Series(v)) for k,v in precip_data.items() ]))
d

Precipitation Data


,meta,smry
lat,"[[36.958333, 36.958333, 36.958333, 36.958333, ...",NaN
lon,"[[-109.083333, -109.041667, -109.0, -108.95833...",NaN
0,NaN,"[[6.06, 6.0, 5.97, 5.94, 5.84, 5.82, 5.72, 5.6..."


In [68]:
print("Temperature and Elevation Data")
d = pandas.DataFrame(dict([ (k,pandas.Series(v)) for k,v in temp_elev_data.items() ]))
d

Temperature and Elevation Data


,meta,smry
0,NaN,"[[101, 101, 101, 101, 101, 101, 101, 101, 101,..."
elev,"[[5062, 4757, 4642, 4810, 4964, 5010, 5046, 51...",NaN
lat,"[[36.958333, 36.958333, 36.958333, 36.958333, ...",NaN
lon,"[[-109.083333, -109.041667, -109.0, -108.95833...",NaN


Note: the lat and lon coordinates line up, so we only need to extract them from one of the jsons.

In [69]:
precip = np.array(precip_data['smry'][0])
elev = np.array(temp_elev_data['meta']['elev'])
lat, lon = np.array(temp_elev_data['meta']['lat']), np.array(temp_elev_data['meta']['lon'])
temp = np.array(temp_elev_data['smry'][0])

In [70]:
print(precip.shape)
print(elev.shape)
print(lat.shape, lon.shape)
print(temp.shape)

(99, 171)
(99, 171)
(99, 171) (99, 171)
(99, 171)


In [91]:
all_data = np.reshape(np.dstack((lat,lon,elev,precip,temp)),(99*171,5))
all_data.shape

(16929, 5)

In [92]:
print(elev[0,0],precip[0,0],temp[0,0])
print(all_data[0])

5062 6.06 101
[  36.958333 -109.083333 5062.          6.06      101.      ]


## K Means

In [95]:
nbrs = NearestNeighbors(n_neighbors=3).fit(all_data)
distances, indices = nbrs.kneighbors(all_data)

In [98]:
nbrs.kneighbors_graph(all_data).toarray().shape

(16929, 16929)

In [97]:
distances

array([[ 0.        ,  3.1336995 ,  3.73199747],
       [ 0.        ,  2.301139  ,  4.24469835],
       [ 0.        ,  4.53756708,  4.81749365],
       ...,
       [ 0.        ,  1.02320309,  4.84403097],
       [ 0.        ,  3.51090404,  5.66227378],
       [ 0.        ,  7.64286191, 12.2946482 ]])